In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [9]:
test_table=pd.read_csv('test.csv')
train_table=pd.read_csv('train.csv')
train_y=train_table.SalePrice
train_x=train_table[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']]
iowa_model = RandomForestClassifier(n_estimators=100, random_state=11)
iowa_model.fit(train_x, train_y)
val_predictions = iowa_model.predict(test_table[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']])
#mean_absolute_error(val_predictions, train_y)

In [10]:
val_predictions

array([151500, 168000, 177000, ..., 157900, 142500, 228000])

In [11]:

df = pd.DataFrame({'Id': test_table.Id, 'SalePrice': val_predictions})
df.to_csv('qwe.csv', encoding='utf-8', index=False)